In [1]:
from audioop import avg
import cmath
from fileinput import filename
from gc import enable
import sys
from matplotlib.pyplot import axis
import pandas as pd
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, roc_auc_score
import warnings
warnings.filterwarnings("ignore")
from fastai.tabular.all import *

dataPath = ('../dataset/UGR/')
df_train = pd.read_csv('../dataset/UGR/train_ori.csv')
df_test = pd.read_csv('../dataset/UGR/test.csv')

y_names = 'Label'

print('Train dataset',"\n",df_train[y_names].value_counts(), "\n")
print('--------------------------------------------------------')
print('Test dataset',"\n",df_test[y_names].value_counts(), "\n")
print('--------------------------------------------------------')

/home/mltest/my_project/my_project_env/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


Train dataset 
 scan44             14093
background         14029
scan11             14016
blacklist          14013
dos                 3165
nerisbotnet         1430
anomaly-spam         266
anomaly-sshscan       75
Name: Label, dtype: int64 

--------------------------------------------------------
Test dataset 
 blacklist          5987
scan11             5984
background         5971
scan44             5907
dos                1335
nerisbotnet         570
anomaly-spam        134
anomaly-sshscan      34
Name: Label, dtype: int64 

--------------------------------------------------------


In [2]:
labels = ['background', 'scan11', 'scan44', 'blacklist', 'nerisbotnet', 'anomaly-spam', 'anomaly-sshscan','dos']

cat_names = ['source_ip', 'dest_ip', 'source_port', 'dest_port', 
             'protocol', 'flag1', 'flag2', 'flag3',
             'flag4', 'flag5', 'flag6']

cont_names = ['hour', 'minute', 'second', 'duration', 'stos', 'pkt','fwd', 'byt']

procs = [Categorify, FillMissing, Normalize]

dls = TabularDataLoaders.from_df(df_train, path=dataPath, procs=procs, y_names=y_names, bs=400) #, valid_idx=list(range(1,test.shape[0])))
roc_auc = RocAuc(average='weighted')
learn = tabular_learner(dls, layers=[400,200], metrics=[accuracy, Precision(average='weighted'), F1Score(average='weighted'), Recall(average='weighted'), roc_auc])
learn.fit(1, 1e-2)

dl = learn.dls.test_dl(df_test, with_labels=True, drop_last=False)
    
print("DNN Predicting...")
start = time.time()
nn_preds, tests, clas_idx = learn.get_preds(dl=dl, with_loss=False, with_decoded=True)
elapsed = time.time() - start
    
print('Printing result of DNN')

##### DNN CM
loss, acc, precision, f1, recall, roc = learn.validate(dl=dl)
interp = ClassificationInterpretation.from_learner(learn)
print("DNN ELAPSED: %f" % (elapsed))
print("Confusion Matrix:\n", interp.confusion_matrix())
print('Accuracy: {:.2f}%; precision: {:.2f}%; F1: {:.2f}%; Recall: {:.2f}%; roc-auc: {:.2f}%; elapsed: {:.2f} s'.format(acc*100, precision*100, f1*100, recall*100, roc*100,  elapsed ))


epoch,train_loss,valid_loss,accuracy,precision_score,f1_score,recall_score,roc_auc_score,time
0,0.424438,0.422001,0.803471,0.813723,0.803284,0.803471,0.955740,00:05


DNN Predicting...


Printing result of DNN


DNN ELAPSED: 2.459562


Confusion Matrix:
 [[  27    0   11   15    0    0    0    0]
 [   0   11    2    0    0    1    0    0]
 [   8    5 1833  960    0   10    3    5]
 [   7    7  764 2063    3    2    4    8]
 [   0    0    3   81  540    3    0    1]
 [   0    1   16  191    0  104    0    0]
 [   0    0    7    1    0    0 2795    7]
 [   0    0   12   41    0    0  222 2443]]
Accuracy: 80.60%; precision: 81.65%; F1: 80.63%; Recall: 80.60%; roc-auc: 95.77%; elapsed: 2.46 s


In [ ]:
import pandas as pd

df = pd.read_csv('../dataset/UGR/original_0.csv')

df['byt'].value_counts()